<a href="https://colab.research.google.com/github/PurviPatel20/Dataset/blob/main/google_palm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:

!pip install -q neo4j-driver
!pip install -q gradio


In [38]:
!pip install -q py2neo

In [39]:

from neo4j import GraphDatabase
import spacy
import gradio as gr

In [40]:
driver = GraphDatabase.driver("bolt://44.203.75.161:7687",
                              auth=("neo4j",
                                    "hoops-formation-ditto"))





In [58]:
with driver.session() as session:
    result = session.run(f"OPTIONAL MATCH (n:ns0__Ubhadi) WHERE n.rdfs__label = 'GG 5' RETURN n.ns0__hasPrice")
    if result.peek() is None:
        print("No information found for the seed GG 5")
    for record in result:
        final = record['n.ns0__hasPrice']
        print(f'The price of seed variety GG 5 is {final}')






The price of seed variety GG 5 is 2310


In [42]:
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship
import pandas as pd

In [44]:
import spacy

In [45]:

# !python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 750.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [47]:
nlp = spacy.load("en_core_web_lg")
nlp
nlp.pipe_labels['ner']


['CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART']

In [48]:
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None


In [49]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history

In [62]:
def get_answer(question):
    doc = nlp(question)
    seed = extract_entity(question, ["PRODUCT","ORG"])

    if seed is None:
        return "Sorry, I don't understand what movie you're asking about."

    with driver.session() as session:
        result = session.run(f"OPTIONAL MATCH (n:ns0__Ubhadi) WHERE n.rdfs__label = 'GG 5' RETURN n.ns0__hasPrice")
        if result.peek() is None:
            return f"No information found for the movie '{seed}'"
        for record in result:
            final = record['n.ns0__hasPrice']
            return f"The price of '{seed}' is: {final}"



In [ ]:

gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8ee716a2e3144a516f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
